In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, when, first, avg
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from functools import reduce
from pyspark.sql import DataFrame

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# Carregar tabelas
time_df = spark.table("database.time")
unemployment_df = spark.table("database.gender")

# Lista de países e anos
paises = ["%Germany%", "%Switzerland%", "%Poland%", "%Hungary%"]
anos = ["2010", "2013"]

# Filtrando os DataFrames

def filter_by_multiple_conditions(df: DataFrame, paises: list, anos: list) -> DataFrame:
    conditions = reduce(lambda a, b: a | b, (col("nome_pais").like(p) for p in paises))
    df = df.filter(conditions)
    df = df.filter(col("ano").isin(anos))
    return df

time_df = filter_by_multiple_conditions(time_df, paises, anos)
unemployment_df = filter_by_multiple_conditions(unemployment_df, paises, anos)

time_df = time_df.drop("cod_indicador", "cod_pais")

unemployment_df = unemployment_df.drop("Country_code", "Country_name", "Indicator_name", "Indicator_code", "cod_indicador", "cod_pais")

# Filtrar pelos indicadores de emprego
unemployment_df = unemployment_df.filter(F.col("nome_indicador").contains("Unemployment") & ~F.col("nome_indicador").contains("with"))

# Adicionar a coluna "genero" com base no nome do indicador
unemployment_df = unemployment_df.withColumn("desagregacao",
    F.when(F.col("nome_indicador").like("% female%"), "female")
    .when(F.col("nome_indicador").like("% male%"), "male")
)
unemployment_df = unemployment_df.filter(F.col("desagregacao").isNotNull())

# Calcular a média dos valores por ano, gênero e indicador
unemployment_df_media = unemployment_df.groupBy("nome_pais", "ano", "desagregacao", "nome_indicador") \
    .agg(F.avg("valor").alias("valor"))

# Selecionar apenas uma linha por ano e gênero
window = Window.partitionBy("nome_pais", "ano", "desagregacao").orderBy(F.col("valor"))

unemployment_df_final = unemployment_df_media.withColumn("row_number", F.row_number().over(window)) \
    .filter(F.col("row_number") == 1).drop("row_number")

columns_order = ["nome_indicador", "nome_pais", "ano", "valor", "desagregacao"]

# Reorganizando as colunas no DataFrame unemployment_df_final
unemployment_df_final = unemployment_df_final.select(columns_order)

In [3]:
time_df = time_df.withColumn("valor", col("valor").cast("float"))
unemployment_df_final = unemployment_df_final.withColumn("valor", col("valor").cast("float"))

In [4]:
df_concatenado = unemployment_df_final.union(time_df)

In [5]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

df_concatenado.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaUnemploymentGold")

In [6]:
spark.sql(
    """
    SELECT * FROM database.UnemploymentGold
    """
).show(500)

+-----------+--------------------+----+--------+------------+
|  nome_pais|      nome_indicador| ano|   valor|desagregacao|
+-----------+--------------------+----+--------+------------+
|    Germany|Unemployment, fem...|2010|     6.5|      female|
|    Germany|Unemployment, mal...|2010|    7.37|        male|
|    Germany|Unemployment, fem...|2013|   4.919|      female|
|    Germany|Unemployment, mal...|2013|   5.498|        male|
|    Hungary|Unemployment, fem...|2010|  10.657|      female|
|    Hungary|Unemployment, mal...|2010|  11.606|        male|
|    Hungary|Unemployment, fem...|2013|   10.14|      female|
|    Hungary|Unemployment, mal...|2013|   10.21|        male|
|     Poland|Unemployment, fem...|2010|   10.02|      female|
|     Poland|Unemployment, mal...|2010|    9.32|        male|
|     Poland|Unemployment, fem...|2013|   11.11|      female|
|     Poland|Unemployment, mal...|2013|    9.69|        male|
|Switzerland|Unemployment, fem...|2010|     5.2|      female|
|Switzer

In [7]:
spark.stop()